# Script para comparar 2 arquivos excel

## Executando 3 comparacoes, dados comuns entre eles, dados exclusivos apenas no 1 arquivo e dados exclusivos apenas no 2 arquivo

### Para um entendimento melhor o codigo sera dividido em etapas

## Dados Exclusivos da Fotografia

In [23]:
import pandas as pd
import sqlite3
from pandasql import sqldf

# Definição do caminho do arquivo
caminho_fotografia = r'C:\Users\luanzera\Desktop\Estudos\Taks BBA - Comparando arquivos\Arquivos p Comparar\Fotografia\Fotografia Operacoes Funded.xlsx'
caminho_baseUnificada = r'C:\Users\luanzera\Desktop\Estudos\Taks BBA - Comparando arquivos\Arquivos p Comparar\BaseUnificada\BaseUnificada.xlsx'

# Leitura dos arquivos excel
fotografia_Funded = pd.read_excel(caminho_fotografia, sheet_name='Op Funded')
Base_Unificada = pd.read_excel(caminho_baseUnificada, sheet_name='BaseUnificada')

# Especificação das colunas desejadas
fotoAtt = ['BANCO', 'Amount', 'Transaction Type', 'Maturity']
baseAtt = ['BANCO', 'AMOUNT_USD', 'TRANSACTION', 'MATURITY']

# Leitura apenas das colunas desejadas do arquivo fotografia_Funded
df_Foto = pd.read_excel(caminho_fotografia, sheet_name='Op Funded', usecols=fotoAtt)
df_Base = pd.read_excel(caminho_baseUnificada, sheet_name='BaseUnificada', usecols=baseAtt)


# Criar conexão com o banco de dados em memória
conn = sqlite3.connect(':memory:')

# Definir a consulta SQL
with conn:
    # Criar a tabela fotografia_Funded
    df_Foto.to_sql('fotografia_Funded', conn, if_exists='replace', index=False)
    
    # Criar a tabela Base_Unificada
    df_Base.to_sql('Base_Unificada', conn, if_exists='replace', index=False)
    
    # Executar a consultas SQLs
    # Join utilizando SQL para comparar as colunas LEFT pela Foto
    query1 = """
        SELECT fotografia_Funded.*,
               Base_Unificada.BANCO AS BANCO_BASE,
               Base_Unificada.AMOUNT_USD,
               Base_Unificada.`TRANSACTION`,
               Base_Unificada.MATURITY
        FROM fotografia_Funded
        LEFT OUTER JOIN Base_Unificada
        ON fotografia_Funded.BANCO = Base_Unificada.BANCO
        AND fotografia_Funded.Amount = Base_Unificada.AMOUNT_USD
        AND fotografia_Funded.`Transaction Type` = Base_Unificada.`TRANSACTION`
        AND fotografia_Funded.Maturity = Base_Unificada.MATURITY
        WHERE Base_Unificada.BANCO IS NULL
        """

    dados_left = pd.read_sql_query(query1, conn)

# Verificar se a coluna "Amount" é NaN e preencher com o valor da coluna "AMOUNT_USD"
dados_left['Amount'] = dados_left['Amount'].fillna(dados_left['AMOUNT_USD'])

# Verificar se a coluna "Transaction Type" é NaN e preencher com o valor da coluna "TRANSACTION"
dados_left['Transaction Type'] = dados_left['Transaction Type'].fillna(dados_left['TRANSACTION'])

# Verificar se a coluna "Maturity" é NaN e preencher com o valor da coluna "MATURITY"
dados_left['Maturity'] = dados_left['Maturity'].fillna(dados_left['MATURITY'])

# Remover a coluna "AMOUNT_USD"
dados_left = dados_left.drop(columns=['AMOUNT_USD'])
dados_left = dados_left.drop(columns=['TRANSACTION'])
dados_left = dados_left.drop(columns=['MATURITY'])
dados_left = dados_left.drop(columns=['BANCO_BASE'])

dados_left


,BANCO,Amount,Transaction Type,Maturity
0,JP MORGAN LUANZERA Foto,180000000,TRADE,2023-12-06 00:00:00
1,DALE DELE BANCO,2342421,TRADE,2023-01-08 00:00:00
2,FOTOGRAFIA BANK,9843532,TRADE,18/12/2023


## Dados Exclusivos da BaseUnificada

In [24]:

    # Join utilizando SQL para comparar as colunas LEFT pela Foto
    query1 = """
        SELECT Base_Unificada.*,
               fotografia_Funded.BANCO AS BANCO_FOTO,
               fotografia_Funded.Amount,
               fotografia_Funded.`Transaction Type`,
               fotografia_Funded.Maturity
        FROM Base_Unificada
        LEFT OUTER JOIN fotografia_Funded
        ON Base_Unificada.BANCO = fotografia_Funded.BANCO
        AND Base_Unificada.AMOUNT_USD = fotografia_Funded.Amount
        AND Base_Unificada.`TRANSACTION` = fotografia_Funded.`Transaction Type`
        AND Base_Unificada.MATURITY = fotografia_Funded.Maturity
        WHERE fotografia_Funded.BANCO IS NULL
        """

    dados_right = pd.read_sql_query(query1, conn)

# Verificar se a coluna "Amount" é NaN e preencher com o valor da coluna "AMOUNT_USD"
dados_right['Amount'] = dados_right['Amount'].fillna(dados_right['AMOUNT_USD'])

# Verificar se a coluna "Transaction Type" é NaN e preencher com o valor da coluna "TRANSACTION"
dados_right['Transaction Type'] = dados_right['Transaction Type'].fillna(dados_right['TRANSACTION'])

# Verificar se a coluna "Maturity" é NaN e preencher com o valor da coluna "MATURITY"
dados_right['Maturity'] = dados_right['Maturity'].fillna(dados_right['MATURITY'])

# Remover a coluna "AMOUNT_USD"
dados_right = dados_right.drop(columns=['AMOUNT_USD'])
dados_right = dados_right.drop(columns=['TRANSACTION'])
dados_right = dados_right.drop(columns=['MATURITY'])
dados_right = dados_right.drop(columns=['BANCO_FOTO'])

dados_right


,BANCO,Amount,Transaction Type,Maturity
0,JP MORGAN da BASE,180000000,TRADE,2023-12-06 00:00:00
1,BANCO LUANZERA,9824732,TRADE,2023-12-06 00:00:00
2,BANCO NICOLAS,23948278,WK,2023-03-09 00:00:00


## Dados comuns entre os arquivos

In [25]:

    # Join utilizando SQL para comparar as colunas INNER
    query3 = """
        SELECT Base_Unificada.*, 
                fotografia_Funded.BANCO AS BANCO_FOTO, 
                fotografia_Funded.Amount, 
                fotografia_Funded.`Transaction Type`, 
                fotografia_Funded.Maturity
        FROM Base_Unificada
            INNER JOIN fotografia_Funded
            ON fotografia_Funded.BANCO = Base_Unificada.BANCO
            AND fotografia_Funded.Amount = Base_Unificada.AMOUNT_USD
            AND fotografia_Funded.`Transaction Type` = Base_Unificada.`TRANSACTION`
            AND fotografia_Funded.Maturity = Base_Unificada.MATURITY
        """

    dados_comuns = pd.read_sql_query(query3, conn)
    
# Removendo Colunas 
dados_comuns = dados_comuns.drop(columns=['Amount'])
dados_comuns = dados_comuns.drop(columns=['Transaction Type'])
dados_comuns = dados_comuns.drop(columns=['Maturity'])
dados_comuns = dados_comuns.drop(columns=['BANCO_FOTO'])

dados_comuns


,BANCO,AMOUNT_USD,TRANSACTION,MATURITY
0,BNY MELLON,18000000,TRADE,2023-02-06 00:00:00
1,DEUTSCHE BANK,27000000,TRADE,2023-06-03 00:00:00
2,HSBC,31000000,WK,2023-12-05 00:00:00
3,YES BANK,22000000,WK,2023-01-08 00:00:00
4,SOCIETE GENERALE,87000000,TRADE,2023-12-05 00:00:00
5,JP MORGAN,70000000,WK,2023-03-09 00:00:00
6,SANTANDER,1000000,WK,2023-01-08 00:00:00
7,BANCOLOMBIA,27030000,STRUCT,2023-12-05 00:00:00
8,BANCO DEL CHILE,73000000,TRADE,2023-03-09 00:00:00
9,CHINA BANK,64000000,TRADE,2023-06-04 00:00:00


#### Todos os dados carregados

In [26]:
import pandas as pd

# Definição do caminho do arquivo
caminho_fotografia = r'C:\Users\luanzera\Desktop\Estudos\Taks BBA - Comparando arquivos\Arquivos p Comparar\Fotografia\Fotografia Operacoes Funded.xlsx'
caminho_baseUnificada = r'C:\Users\luanzera\Desktop\Estudos\Taks BBA - Comparando arquivos\Arquivos p Comparar\BaseUnificada\BaseUnificada.xlsx'
# Leitura dos arquivos excel
fotografia_Funded = pd.read_excel(caminho_fotografia, sheet_name='Op Funded')
Base_Unificada = pd.read_excel(caminho_baseUnificada, sheet_name='BaseUnificada')

# Especificação das colunas desejadas
fotoAtt = ['BANCO', 'Amount', 'Transaction Type', 'Maturity']
baseAtt = ['BANCO', 'AMOUNT_USD', 'TRANSACTION', 'MATURITY']

# Leitura apenas das colunas desejadas do arquivo fotografia_Funded
df_Foto = pd.read_excel(caminho_fotografia, sheet_name='Op Funded', usecols=fotoAtt)
df_Base = pd.read_excel(caminho_baseUnificada, sheet_name='BaseUnificada', usecols=baseAtt)

# Realizar o merge dos dataframes com base na coluna "BANCO"
df_merged = pd.merge(df_Foto, df_Base, on='BANCO', how='outer')

# Verificar se a coluna "Amount" é NaN e preencher com o valor da coluna "AMOUNT_USD"
df_merged['Amount'] = df_merged['Amount'].fillna(df_merged['AMOUNT_USD'])

# Verificar se a coluna "Transaction Type" é NaN e preencher com o valor da coluna "TRANSACTION"
df_merged['Transaction Type'] = df_merged['Transaction Type'].fillna(df_merged['TRANSACTION'])

# Verificar se a coluna "Maturity" é NaN e preencher com o valor da coluna "MATURITY"
df_merged['Maturity'] = df_merged['Maturity'].fillna(df_merged['MATURITY'])

# Remover a coluna "AMOUNT_USD"
df_merged = df_merged.drop(columns=['AMOUNT_USD'])
df_merged = df_merged.drop(columns=['TRANSACTION'])
df_merged = df_merged.drop(columns=['MATURITY'])

df_merged


,BANCO,Amount,Transaction Type,Maturity
0,BNY MELLON,18000000.0,TRADE,2023-02-06 00:00:00
1,JP MORGAN LUANZERA Foto,180000000.0,TRADE,2023-12-06 00:00:00
2,DEUTSCHE BANK,27000000.0,TRADE,2023-06-03 00:00:00
3,HSBC,31000000.0,WK,2023-12-05 00:00:00
4,HSBC,31000000.0,WK,2023-12-05 00:00:00
5,HSBC,52000000.0,TRADE,2023-10-03 00:00:00
6,HSBC,52000000.0,TRADE,2023-10-03 00:00:00
7,YES BANK,22000000.0,WK,2023-01-08 00:00:00
8,SOCIETE GENERALE,87000000.0,TRADE,2023-12-05 00:00:00
9,JP MORGAN,70000000.0,WK,2023-03-09 00:00:00
